In [161]:
import pandas as pd
import calendar

def get_snotel_df(file='../content/csv/snotel/new_datasets/1012_new_metrics.csv'):
    df = pd.read_csv(file, comment='#')
    df = df.set_index('Date')
    # Rename the columns to be the same as listed below
    df.columns.values[0] = 'Max Air Temp 24hr'
    df.columns.values[1] = 'Total Snowfall 24hr'
    df.columns.values[2] = 'Min Temp Diff 48hr'
    df.columns.values[3] = 'Delta SWE 24hr'
    
    return df

def get_danger_df(area='West Slopes South', drop_no_rating=True):
    def transform_date(date_string):
        parts = date_string.split(' ')
        month = parts[0]
        day = parts[1]
        year = parts[2]

        day = '0' + day if len(day) < 2 else day
        month = list(calendar.month_abbr).index(month)
        month = '0' + str(month) if month < 10 else month
        return f'{year}-{month}-{day}'

    df = pd.read_csv('../content/csv/danger_ratings.csv')
    if drop_no_rating:
        df = df[df['Danger Rating'] != 'NO RATING']
    df = df[(df == area).any(axis=1)]
    df['Danger Rating'] = df['Danger Rating'].map({ 'NO RATING': 0, 'LOW': 1, 'MODERATE': 2, 'CONSIDERABLE': 3, 'HIGH': 4, 'EXTREME': 5 })
    try:
        df['Date'] = df['Date'].apply(transform_date) # 2022-11-01
    except Exception as err:
        print('Cant format date:', err)

    df = df.set_index('Date')
    return df


def get_training_data():
    snotel_df = get_snotel_df()
    danger_df = get_danger_df()
    
    def init_training_df():
        cols = [ 
            'Max Air Temp 24hr', 
            'Max Air Temp 72hr', 
            'Total Snowfall 24hr', 
            'Total Snowfall 72hr', 
            'Max Windspeed 24hr',
            'Weighted Snowfall 96hr',
            'Min Temp Diff 48hr',
            'Was Heavy Snowfall 24hr',
            'Was High Winds 24hr',
            'Sum Max Temp 72hr',
            'Delta SWE 24hr',
            'Yesterday Danger',
            'Danger Rating'
        ]
        return pd.DataFrame([], columns=cols)
    
    # Set up the correct columns
    df = init_training_df()
    # For every danger rating value
    for idx, row in danger_df.iterrows():
        date = idx
        # Compute the metrics for that date
        
        # Add them to a row and add that row to the dataframe
        df.loc[date] = [0,0,0,0,0,0,0,0,0,0,0,0,0]

    return df

print('SWE: Depth of water that would theoretically result if the entire snowpack were melted instantaneously')
print('SD: Total snow depth')
print('PA: Water year accumulated precipitation')
print('ATO: Instantaneously observed air temperature')

print('SNOTEL datasets to use: 418_new_metrics.csv, 804_new_metrics.csv, 941_new_metrics.csv, 1012_new_metrics.csv')

get_danger_df().tail()
# get_training_data().head()

SWE: Depth of water that would theoretically result if the entire snowpack were melted instantaneously
SD: Total snow depth
PA: Water year accumulated precipitation
ATO: Instantaneously observed air temperature
SNOTEL datasets to use: 418_new_metrics.csv, 804_new_metrics.csv, 941_new_metrics.csv, 1012_new_metrics.csv


,Area,Danger Rating
Date,,
2022-11-29,West Slopes South,3
2022-11-28,West Slopes South,2
2022-11-27,West Slopes South,2
2022-11-26,West Slopes South,3
2022-11-25,West Slopes South,1


This^ is what we are hoping to achieve

todo: start here

In [162]:
get_danger_df().head()

,Area,Danger Rating
Date,,
2023-04-15,West Slopes South,2
2023-04-14,West Slopes South,1
2023-04-13,West Slopes South,2
2023-04-12,West Slopes South,2
2023-04-11,West Slopes South,2


Here is an example dataset with the values I think we need. 
1. Calculate additional fields for this dataset (see above)
1. Combine this dataset with the other 3
    - mean
    - median
    - max

In [163]:
get_snotel_df().head()

,Max Air Temp 24hr,Total Snowfall 24hr,Min Temp Diff 48hr,Delta SWE 24hr
Date,,,,
2022-11-01,34.3,0,-0.1,-5.1
2022-11-02,34.2,2,0.4,-4.1
2022-11-03,32.0,3,0.4,2.0
2022-11-04,44.2,7,1.1,5.2
2022-11-05,36.7,-8,0.0,-3.8


Calculate additional fields for this dataset:

We currently have: 
- 'Max Air Temp 24hr' 
- 'Total Snowfall 24hr'
- 'Min Temp Diff 48hr'
- 'Delta SWE 24hr'


This dataset can't do wind or danger (yet). But we CAN calculate:
- 'Max Air Temp 72hr'
- 'Total Snowfall 72hr'
- 'Weighted Snowfall 96hr'
- 'Was Heavy Snowfall 24hr'
- 'Sum Max Temp 72hr'

In [164]:
def take_snotel_df_and_calculate_columns(df):
    """Using raw SNOTEL data, calculate the above columns and return them as a dataframe"""

    # Add the column placeholders to the dataframe (or initialize a new one); initialize as null for now
    # String shorthand
    max_temp_72 = 'Max Air Temp 72hr'
    tot_snow_72 = 'Total Snowfall 72hr'
    wgt_snow_72 = 'Weighted Snowfall 96hr'
    was_hvys_24 = 'Was Heavy Snowfall 24hr'
    sum_mtmp_72 = 'Sum Max Temp 72hr'
    df[max_temp_72] = 0 # None
    df[tot_snow_72] = 0 # None
    df[wgt_snow_72] = 0 # None
    df[was_hvys_24] = 0 # None
    df[sum_mtmp_72] = 0 # None

    # Calculate the above metrics
    one_day_ago = None # This wouldn't be necessary if I wasn't indexing on date
    two_days_ago = None
    three_days_ago = None
    for idx, day in df.iterrows():
        df.at[idx, was_hvys_24] = 1 if day['Total Snowfall 24hr'] >= 12 else 0 # todo: update threshold
        if two_days_ago is not None:
            df.at[idx, max_temp_72] = max(
                day['Max Air Temp 24hr'], 
                one_day_ago['Max Air Temp 24hr'], 
                two_days_ago['Max Air Temp 24hr'])
            df.at[idx, tot_snow_72] = \
                day['Total Snowfall 24hr'] \
                + one_day_ago['Total Snowfall 24hr'] \
                + two_days_ago['Total Snowfall 24hr']
            df.at[idx, sum_mtmp_72] = \
                day['Max Air Temp 24hr'] \
                + one_day_ago['Max Air Temp 24hr'] \
                + two_days_ago['Max Air Temp 24hr'] 
        if three_days_ago is not None:
            df.at[idx, 'Weighted Snowfall 96hr'] = \
                day['Total Snowfall 24hr'] * 1.0 \
                + one_day_ago['Total Snowfall 24hr'] * 0.75 \
                + two_days_ago['Total Snowfall 24hr'] * 0.5 \
                + three_days_ago['Total Snowfall 24hr'] * 0.25

        # Update past day placeholders
        three_days_ago = two_days_ago
        two_days_ago = one_day_ago
        one_day_ago = day
    
    # Return the new/updated dataframe
    return df

take_snotel_df_and_calculate_columns(get_snotel_df()).head()


,Max Air Temp 24hr,Total Snowfall 24hr,Min Temp Diff 48hr,Delta SWE 24hr,Max Air Temp 72hr,Total Snowfall 72hr,Weighted Snowfall 96hr,Was Heavy Snowfall 24hr,Sum Max Temp 72hr
Date,,,,,,,,,
2022-11-01,34.3,0,-0.1,-5.1,0.0,0,0.00,0,0.0
2022-11-02,34.2,2,0.4,-4.1,0.0,0,0.00,0,0.0
2022-11-03,32.0,3,0.4,2.0,34.3,5,0.00,0,100.5
2022-11-04,44.2,7,1.1,5.2,44.2,12,10.25,0,110.4
2022-11-05,36.7,-8,0.0,-3.8,44.2,2,-0.75,0,112.9


Combine our new datasets

In [165]:
from os import listdir
from os.path import isfile, join

def get_concat_snotel():
     # Load the 4 snotel datasets with extra columns created as above
    path = '../content/csv/snotel/new_datasets/'
    files = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    dataframes = [get_snotel_df(file=f) for f in files]
    formatted = [take_snotel_df_and_calculate_columns(df) for df in dataframes]

    # Combine them and get the average of all numerical values
    concat = pd.concat(formatted).groupby(level=0)

    return concat

def snotel_datasets_combined_mean(concat=get_concat_snotel()):
    """Calculate columns for all SNOTEL datasets as above and aggregate them using their average"""
    return concat.mean()

def snotel_datasets_combined_median(concat=get_concat_snotel()):
    """Calculate columns for all SNOTEL datasets as above and aggregate them using their median"""
    return concat.median()

def snotel_datasets_combined_max(concat=get_concat_snotel()):
    """Calculate columns for all SNOTEL datasets as above and aggregate them using their maximum"""
    return concat.max()

snotel_datasets_combined_mean().head()
# snotel_datasets_combined_median().head()
# snotel_datasets_combined_max().head()

,Max Air Temp 24hr,Total Snowfall 24hr,Min Temp Diff 48hr,Delta SWE 24hr,Max Air Temp 72hr,Total Snowfall 72hr,Weighted Snowfall 96hr,Was Heavy Snowfall 24hr,Sum Max Temp 72hr
Date,,,,,,,,,
2022-11-01,35.725,0.25,0.000,-4.325,0.000,0.00,0.0000,0.0,0.000
2022-11-02,34.650,0.75,0.050,-3.400,0.000,0.00,0.0000,0.0,0.000
2022-11-03,34.925,1.25,0.150,0.000,36.025,2.25,0.0000,0.0,105.300
2022-11-04,44.550,4.50,0.800,7.150,44.550,6.50,5.8750,0.0,114.125
2022-11-05,36.450,-4.25,-0.025,-4.675,44.550,1.50,-0.0625,0.0,115.925


Next, do the same thing with the NWAC data to get wind information:
- 'Max Windspeed 24hr'
- 'Was High Winds 24hr'

In [166]:
def take_nwac_df_and_calculate_columns():
    """Using raw NWAC data, calculate the above columns and return them as a dataframe"""
    pass

In [167]:
def nwac_datasets_combined_mean():
    """Calculate columns for all NWAC datasets as above and aggregate them using their average"""
    # Load the nwac datasets with extra columns created

    # Combine them and get the average of all numerical values

    # Return the newly created dataframe
    return None

def nwac_datasets_combined_median():
    """Calculate columns for all NWAC datasets as above and aggregate them using their median"""
    pass

def nwac_datasets_combined_max():
    """Calculate columns for all NWAC datasets as above and aggregate them using their max"""
    pass

In [170]:
def combine_snotel_and_nwac_into_result_dataframe():
    """Join the aggregated SNOTEL, aggregated NWAC, and danger ratings datasets on their Date column"""
    pass

# Placeholder until above is finished
def combine_snotel_and_danger(snotel_df=snotel_datasets_combined_mean(), danger_df=get_danger_df()):
    return danger_df.drop('Area', axis=1).join(snotel_df)

combine_snotel_and_danger().tail()


,Danger Rating,Max Air Temp 24hr,Total Snowfall 24hr,Min Temp Diff 48hr,Delta SWE 24hr,Max Air Temp 72hr,Total Snowfall 72hr,Weighted Snowfall 96hr,Was Heavy Snowfall 24hr,Sum Max Temp 72hr
Date,,,,,,,,,,
2023-04-11,2,38.300,-2.00,0.425,-6.025,44.600,-5.25,-4.7500,0.0,124.075
2023-04-12,2,39.000,2.75,0.050,0.800,42.200,-1.75,-0.1875,0.0,119.225
2023-04-13,2,38.925,-1.25,-0.100,-0.600,39.475,-0.50,-0.8125,0.0,116.225
2023-04-14,1,46.225,-0.25,-0.100,0.200,46.225,1.25,-0.3125,0.0,124.150
2023-04-15,2,46.275,-0.75,-0.175,8.350,46.775,-2.25,-0.8750,0.0,131.425
